In [2]:
# carregar base de dados
from test_helper import Test
import os.path
baseDir = os.path.join('Data')
inputPath = os.path.join('Aula04', 'BikeSharing.csv')
fileName = os.path.join(baseDir, inputPath)

numPartitions = 2
csvData = sc.textFile(fileName, numPartitions)
header = csvData.take(1)[0]
rawData = csvData.filter(lambda x: x!=header)

'''
https://www.kaggle.com/c/bike-sharing-demand
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals
'''

'\nhttps://www.kaggle.com/c/bike-sharing-demand\ndatetime - hourly date + timestamp  \nseason -  1 = spring, 2 = summer, 3 = fall, 4 = winter \nholiday - whether the day is considered a holiday\nworkingday - whether the day is neither a weekend nor holiday\nweather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy \n2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist \n3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds \n4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog \ntemp - temperature in Celsius\natemp - "feels like" temperature in Celsius\nhumidity - relative humidity\nwindspeed - wind speed\ncasual - number of non-registered user rentals initiated\nregistered - number of registered user rentals initiated\ncount - number of total rentals\n'

In [3]:

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import DenseVector
from datetime import datetime as dt
import numpy as np
import math

def discrete(value, max, n):
    """Divide o intervalo de 0 até 'max' em 'n' intervalos de mesmo tamanho e retorna qual intervalo 'value' corresponde.
    
    Args:
        value: valor a ser discretizado.
        max: valor maximo que 'value' pode assumir.
        n: número de classes que serão levadas
    """
    
    if value==max: value-=1
    gap=max/n
    return int(math.floor(value/gap))

    

def parsePoint(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    data = point.split(',')
    Date = dt.strptime(data[0], "%Y-%m-%d %H:%M:%S")
    DateList = [Date.year-2011, Date.month, Date.hour]   
    season = data[1]
    workingday=data[3]
    weather =data[4]
    sense=discrete(float(data[6]),46.,8)
    hum=discrete(float(data[7]),100.,8)
    wind=discrete(float(data[8]),57.,8)
    #features = DenseVector(DateList + season + realValues)
    features = DenseVector(DateList + [season]+ [workingday] + [sense] + [hum] + [wind] + [weather])# + realValues)
    
    label = discrete(int(data[-1]),997,10)
    
    return LabeledPoint(label, features)

weights = [.8, .1, .1]
seed = 42

rawTrainData, rawValData, rawTestData = rawData.randomSplit(weights, seed)

parsedTrainData = rawTrainData.map(parsePoint).cache()
parsedValData = rawValData.map(parsePoint).cache()
parsedTestData = rawTestData.map(parsePoint).cache()

print parsedTrainData.count()

8761


In [4]:
#Função própria do Spark
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel

# Train a naive Bayes model.
model = NaiveBayes.train(parsedTrainData, 1.0)

# Make prediction and test accuracy.
predictionAndLabel = parsedTestData.map(lambda p : (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / parsedTestData.count()
print accuracy

0.426679280984


In [21]:
#Fiz os dados do desafio 1 funcionarem com o Bayes, só falta começar agora.
import numpy as np

#Para cada classe c, calcular P(c). Para cada valor de atributo a, calcular P(a|c)
RDDlen=parsedTrainData.count()

#Probabilidade de cada classe
PClassesRDD=(parsedTrainData
                .map(lambda lp:(lp.label,1))
                .reduceByKey(lambda x,y:x+y)
                .map(lambda (x,y):(x,np.log10(y/float(RDDlen))))
            )

#Probabilidade de cada classe, sem logartimo, para uso na probabilidade condicional 
PClassesWithoutlog=(parsedTrainData
                .map(lambda lp:(lp.label,1))
                .reduceByKey(lambda x,y:x+y)
                .map(lambda (x,y):(x,y/float(RDDlen)))
            )
PClassesWithoutlog = PClassesWithoutlog.collectAsMap()

#Probabilidade de cada atributo
#Tiramos a quantidade de atributos de uma amostra
amostra = parsedTrainData.take(1)[0]

#quantidade de atributos
nAttri=len(amostra.features)
#lista de RDDs para cada atributo
AttributesRDDs=[]
#lista de RDDs para probabilidade de cada atributo dado classe
AttributeClassesRDDs=[]
#laço para percorrer os atributos
for i in range(nAttri):
    AttriRDD =(parsedTrainData
                .map(lambda lp:(lp.features[i],1))
                .reduceByKey(lambda x,y:x+y)
                .map(lambda (x,y):(x,np.log10(y/float(RDDlen)))))
    AttributesRDDs.append(AttriRDD)
    AttriClassRDD=(parsedTrainData
                    .map(lambda lp:((lp.label,lp.features[i]),1))
                    .reduceByKey(lambda x,y:x+y)
                    .map(lambda (x,y) : (x, y/float(RDDlen)))
                    .map(lambda (x,y) : (x, np.log10(y/PClassesWithoutlog[x[1]]))) 
                  )
    AttributeClassesRDDs.append(AttriClassRDD)
    


[((0.0, 0.0), -0.22927639553818416), ((9.0, 1.0), -2.1668625760868299), ((7.0, 1.0), -1.303082323104819), ((3.0, 1.0), -0.58183592405764806), ((6.0, 0.0), -2.495165757413305), ((2.0, 0.0), -0.77992233432568203), ((5.0, 1.0), -0.88635424756745063), ((1.0, 1.0), -0.39544470836215484), ((4.0, 0.0), -1.2600966383982861), ((0.0, 1.0), -0.13127891463931898)]


In [22]:
import numpy as np
def naiveBayesTrain(trainSet):
    """Recebe um set de treinos e calcula probabilidades e cria um modelo

    Args:
        trainSet (list)->(label, features)

    Returns:
        list: Um model, array[
                                Dict class : probabilidade(class),
                                RDD de tuplas (atributo, probabilidade(atributo))
                                RDD de tuplas ((class, atributo), probabilidade(atributo | class))
                                Int Quantidade de classes
                             ]
    """
    RDDlen=trainSet.count()
    #Probabilidade de cada classe
    PClassesRDD=(trainSet
                .map(lambda lp:(lp.label,1))
                .reduceByKey(lambda x,y:x+y)
                .map(lambda (x,y):(x,np.log10(y/float(RDDlen))))
                )
    #Podemos dar colect pra virar um dict, e facilitar o acesso, numero de classes nao sera grande suficiente para estourar
    PClassesRDD = PClassesRDD.collectAsMap()
    #Probabilidade de cada classe, sem logartimo, para uso na probabilidade condicional 
    PClassesWithoutlog=(trainSet
                                .map(lambda lp:(lp.label,1))
                                .reduceByKey(lambda x,y:x+y)
                                .map(lambda (x,y):(x,y/float(RDDlen)))
                        )
    PClassesWithoutlog = PClassesWithoutlog.collectAsMap()
    
    #Probabilidade de cada atributo
    #Tiramos a quantidade de atributos de uma amostra
    amostra = trainSet.take(1)[0]

    #quantidade de atributos
    nAttri=len(amostra.features)
    #lista de RDDs para cada atributo
    AttributesRDDs=[]
    #lista de RDDs para probabilidade de cada atributo dado classe
    AttributeClassesRDDs=[]
    
    #laço para percorrer os atributos
    for i in range(nAttri):
        AttriRDD =(trainSet
                    .map(lambda lp:(lp.features[i],1))
                    .reduceByKey(lambda x,y:x+y)
                    .map(lambda (x,y):(x,np.log10(y/float(RDDlen))))
                  )
        AttributesRDDs.append(AttriRDD)
        AttriClassRDD=(trainSet
                        .map(lambda lp:((lp.label,lp.features[i]),1))
                        .reduceByKey(lambda x,y:x+y)
                        .map(lambda (x,y) : (x, y/float(RDDlen)))
                        .map(lambda (x,y) : (x, np.log10(y/PClassesWithoutlog[x[1]]))) 
                      )
        AttributeClassesRDDs.append(AttriClassRDD)
        
    return [PClassesRDD, AttributesRDDs, AttributeClassesRDDs, nAttri]

def predict(model, objectToPredict):
    Pc = model[0]
    AttributesRDDs = model[1]
    AttributeClassesRDDs = model[2]
    nAttri = model[3]
    sumClass = -1
    bestClass = 0
    Pa = []
    Pac = []
    #Filtrar RDD de probabilidade para conter somente as que sao necessarias para o objeto a ser predito e entao criar um dict
    for i in range(nAttri):
        Pa.append(AttributesRDDs[i].filter(lambda x : x[0] in objectToPredict.features).collectAsMap())
        Pac.append(AttributeClassesRDDs[i].filter(lambda x : x[0][1] in objectToPredict.features).collectAsMap())
    
    ### AJUDEM AQUI
    #Pa e Pac sao lists de dict, como converter para somente 1 dict?
    
    for i in Pc:
        soma = sum(Pac[(i, a)] + Pc[i] - Pa[a] for a in objectToPredict.features)
        if(soma > sumClass):
            sumClass = soma
            bestClass = i
    return bestClass
        